In [1]:
import sympy as sy
import numpy as np
from IPython.display import display
import sys
sys.setrecursionlimit(100000)


from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

SyntaxError: invalid syntax (sparse.py, line 566)

In [ ]:
PPTY_PHY_PARAMS = {"positive": True, "real" : True }
PPTY_STATE_VAR  = {"real" : True }

class LDL_inv():
    def __init__(self, N, simplify=False):
        self.N = N          # dimension des matrices de contraintes
        self.Nx = self.N +1 # nombre de variables d'états
        
        # On veut inverser le résultat d'une décomposition L.T*D*L
        #    - D est une matrice diagonale
        #    - L est triangulaire inférieure avec diagonale identité
        # Liste des coefficient Dk: 
        self.Di_array  = [None for i in range(self.N)]
        self.L         = sy.eye(N)
        self.Linv      = sy.eye(N)
        self.A         = sy.zeros(N)
        self.mui_array = [None for i in range(self.Nx)]
        self.SIMPLIFY = simplify
        self.oneHalf = sy.Rational(1,2)
    
        self.init_A()
        
        # Initial element of the serie
        self.Dk1 = self.get_mui(1) + self.get_mui(2)
        self.Di_array[0] = self.Dk1
        
    def init_A(self):
        for i in range(self.N):
            self.A[i,i]   = self.get_mui(i+1) + self.get_mui(i+2)
            try: 
                self.A[i+1,i] = self.oneHalf *self.get_mui(i+2)
            except IndexError:
                pass
            try: 
                self.A[i,i+1] = self.oneHalf *self.get_mui(i+2)
            except IndexError:
                pass
             
    
    def create_mui(self, i):
        ''' creates the sympy symbol mu_i = m_i/(ell_i**2)'''
        self.mui_array[i-1] = sy.symbols('\\mu_{}'.format(i),
                            **PPTY_PHY_PARAMS) ## pptés du symb
    
    def create_Di(self, i, form=1, simplify=False):
        '''Recursively create Di: form1 or 2'''
        if form == 1:
            self.Di_array[i-1] = self.get_mui(i) + self.get_mui(i+1) - \
                                (self.oneHalf*self.get_mui(i))**2/self.get_Di(i-1)
        elif form == 2:
            self.Di_array[i-1] = (self.get_Di(i-1)*(self.get_mui(i) + self.get_mui(i+1)) - \
                                (self.oneHalf*self.get_mui(i))**2)/self.get_Di(i-1)
        
        if simplify:
            self.Di_array[i-1] = self.Di_array[i-1].simplify()
            
    def create_Lij(self, i, j, simplify=False):
        '''
        Returns element of row i and col j
        '''
        self.L[i-1,j-1] = self.oneHalf*self.get_mui(i)/self.get_Di(i-1)
    
    def create_Linv_ij(self, i, j, simplify=False):
        '''
        '''
        prod = 1
        for ind in range(j+1,i+2):
            print(ind)
            prod *= -get_mui(ind)/get_Di(ind-1)
        if simplify:
            prod = prod.simplify()
        
        self.Linv[i-1, j-1] = prod
        
    
    #### --- GETTERS --- ###
    def verify_index(self, i, N):
        '''verifies that i<N'''
        assert (i <= N),"Choose a valid index, must be less or equal than {}".format(self.N)
        assert (i > 0), "Index must be higher than 1"
        return True
        
    def get_mui(self, i):
        ''' 
        Returns the sympy symbol mu_i = m_i/(ell_i**2), 
        creates it if needed
        '''
        if self.verify_index(i, self.Nx):
            if self.mui_array[i-1] is None:
                self.create_mui(i)
                return self.mui_array[i-1]
            else:
                return self.mui_array[i-1]

    def get_Di(self, i):
        ''' Returns the i-th element of diagonal matrix D'''
        if self.verify_index(i, self.N):
            if i == 1:
                return self.Dk1
            elif self.Di_array[i-1] is None:
                self.create_Di(i, simplify=self.SIMPLIFY)
                return self.Di_array[i-1]
            else:
                return self.Di_array[i-1]
    
    def get_Lij(self, i, j):
        if self.verify_index(i, self.N) and self.verify_index(j, self.N):
            if(i == j):
                return 1
            if j > i or j < i-1:
                print('Lij_return zero')
                return 0
            elif self.L[i-1,j-1] == 0:
                self.create_Lij(i,j, simplify=self.SIMPLIFY)
                return self.L[i-1,j-1]
            else:
                return self.L[i-1,j-1]
    
    def get_Aij(self, i,j):
        return self.A[i+1, j+1]
    
    def get_Linv_ij(self, i, j):
        if self.verify_index(i, self.N) and self.verify_index(j, self.N):
            if(i == j):
                return 1
            if j>i:
                return 0
            elif self.Linv[-1,-1] == 0:
                self.create_Linv_ij(i, j)
                return self.Linv[i-1,j-1]
            else: 
                return self.Linv[i-1,j-1]

                
    
LDL_inv = LDL_inv(5, simplify=False)
LDL_inv.get_Lij(2,1)
LDL_inv.get_Lij(3,2)
LDL_inv.get_Lij(4,3)
LDL_inv.get_Lij(5,4)
display(LDL_inv.L)


In [ ]:
display(sy.Matrix(LDL_inv.Di_array))

In [ ]:
dim = 3
PPTY = {'real':True, 'positive':True}
Q = sy.zeros(dim)
# --- Construction de la matrice
for i in range(dim):
    label  = str(i) + str(i)
    Q[i,i] = sy.symbols('q_' + label, **PPTY)
    
    label  = str(i+1) + str(i+2)
    try:
        Q[i,i+1] = sy.symbols('q_' + label, **PPTY)
    except IndexError:
            pass
    label  = str(i) + str(i+1)
    try:
        Q[i,i-1] = sy.symbols('q_' + label, **PPTY)
    except IndexError:
        pass
Q[0,-1] = 0
Q = sy.SparseMatrix(Q)

rhs = Q.eye(Q.rows)

# --- Décomposition LDL
L, D = Q.LDLdecomposition()

# --- On touche pas
Y    = L.lower_triangular_solve(rhs)
Z    = D.diagonal_solve(Y)

res2 = (L.T).upper_triangular_solve(Z)

In [ ]:
res2[0,0]